In [1]:
import pandas as pd
import os
import numpy as np
from termcolor import colored
import seaborn as sns
import matplotlib.pyplot as plt
import html
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
df = pd.read_csv('full_merged_dataset15.csv', encoding='utf-8')

In [3]:
df.head()

,id,title,author,snippet,link,categories,poem
0,162275,The 80&rsquo;s Miracle Diet,By Melvin Dixon,Yours free without the asking,https://www.poetryfoundation.org/poetrymagazin...,"['Living', 'Health & Illness']",Yours free without the asking Quick delivery v...
1,162250,All Saints&rsquo;,By Corey Van Landingham,CaravaggioÃ¢â‚¬â„¢s face in the sunken pumpkin.,https://www.poetryfoundation.org/poetrymagazin...,[],CaravaggioÃ¢â‚¬â„¢s face in the sunken pumpkin...
2,162279,And These Are Just a Few ...,By Melvin Dixon,This poem is for the epidemic dead and the liv...,https://www.poetryfoundation.org/poetrymagazin...,"['Living', 'Health & Illness', 'Social Comment...",This poem is for the epidemic dead and the liv...
3,162248,ASMR,By Corey Van Landingham,Why not climb up the mountain,https://www.poetryfoundation.org/poetrymagazin...,[],Why not climb up the mountain of delight? To t...
4,162249,ASMR,By Corey Van Landingham,HelloÃ¢â‚¬â€Tonight,https://www.poetryfoundation.org/poetrymagazin...,[],HelloÃ¢â‚¬â€Tonight weÃ¢â‚¬â„¢ll trace the st...


In [4]:
import re
import nltk

def check_nltk_packages():
  packages = ['punkt','stopwords','omw-1.4','wordnet']

  for package in packages:
    try:
      nltk.data.find('tokenizers/' + package)
    except LookupError:
      nltk.download(package)
check_nltk_packages()

try:
  import lxml
except ModuleNotFoundError:
  %pip install lxml

try:
  import contractions
except ModuleNotFoundError:
  %pip install contractions
  import contractions

from bs4 import BeautifulSoup
import re

[nltk_data] Downloading package stopwords to /home/vm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/vm/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:

def separate_language(df, text_column):
    """
    Separate observations in a DataFrame based on the language detected in a text column.

    Args:
        df (pandas.DataFrame): Input DataFrame.
        text_column (str): Name of the column containing text data.

    Returns:
        tuple: A tuple containing two DataFrames. The first DataFrame contains observations in English,
               and the second DataFrame contains observations in other languages.
    """
    # Initialize lists to store data
    english_data = []
    other_language_data = []

    # Loop through each row in the dataset
    for index, row in df.iterrows():
        text = row[text_column]
        try:
            # Detect language of the text
            lang = detect(text)
        
            # Append entire row data to respective lists
            if lang == 'en':
                english_data.append(row)
            else:
                other_language_data.append(row)
        except:
            # Handle exceptions in language detection (e.g., short texts)
            pass

    # Create dataframes from lists
    english_df = pd.DataFrame(english_data)
    other_language_df = pd.DataFrame(other_language_data)

    return english_df, other_language_df

In [6]:
# Call the function to separate language
english_df, other_language_df = separate_language(df, 'poem')

In [7]:
# i want to check the size of both dfs
print(english_df.shape)
print(other_language_df.shape)

(44869, 7)
(419, 7)


In [8]:
english_df.head()


,id,title,author,snippet,link,categories,poem
0,162275,The 80&rsquo;s Miracle Diet,By Melvin Dixon,Yours free without the asking,https://www.poetryfoundation.org/poetrymagazin...,"['Living', 'Health & Illness']",Yours free without the asking Quick delivery v...
1,162250,All Saints&rsquo;,By Corey Van Landingham,CaravaggioÃ¢â‚¬â„¢s face in the sunken pumpkin.,https://www.poetryfoundation.org/poetrymagazin...,[],CaravaggioÃ¢â‚¬â„¢s face in the sunken pumpkin...
2,162279,And These Are Just a Few ...,By Melvin Dixon,This poem is for the epidemic dead and the liv...,https://www.poetryfoundation.org/poetrymagazin...,"['Living', 'Health & Illness', 'Social Comment...",This poem is for the epidemic dead and the liv...
3,162248,ASMR,By Corey Van Landingham,Why not climb up the mountain,https://www.poetryfoundation.org/poetrymagazin...,[],Why not climb up the mountain of delight? To t...
4,162249,ASMR,By Corey Van Landingham,HelloÃ¢â‚¬â€Tonight,https://www.poetryfoundation.org/poetrymagazin...,[],HelloÃ¢â‚¬â€Tonight weÃ¢â‚¬â„¢ll trace the st...


In [9]:
other_language_df.head()

,id,title,author,snippet,link,categories,poem
10,162260,Fosfor,By Kacper Bartczak,w terrarium jesteÃ…â€º,https://www.poetryfoundation.org/poetrymagazin...,[],w terrarium jesteÃ…â€º czarna skrzyneczka rzec...
29,162261,PieÃ…â€ºÃ…â€ž tego,By Kacper Bartczak,perÃ…â€šo Ã…â€ºwiata wymiocino wieprzu jestem,https://www.poetryfoundation.org/poetrymagazin...,[],perÃ…â€šo Ã…â€ºwiata wymiocino wieprzu jestem ...
32,162263,Prawda obrazu,By Kacper Bartczak,Kobieta naprzeciwko czyta,https://www.poetryfoundation.org/poetrymagazin...,[],Kobieta naprzeciwko czyta Ã¢â‚¬Å¾NajwiÃ„â„¢ksz...
35,162257,regenportr&auml;t,By Jan Wagner,"erscheint als handkuÃƒÅ¸, als sanfter stubser,",https://www.poetryfoundation.org/poetrymagazin...,['Nature'],"erscheint als handkuÃƒÅ¸, als sanfter stubser,..."
37,162256,reifen,By Jan Wagner,"dort, wo die stadt versickert,",https://www.poetryfoundation.org/poetrymagazin...,"['Social Commentaries', 'Cities & Urban Life']","dort, wo die stadt versickert, beim bahndamm, ..."


### Save the english DataFrame and the rest of the data

In [10]:
english_df.to_csv('english_df.csv', index=False) # CAREFUL, you can overwrite your previous dataset, UPDATE the name when used
other_language_df.to_csv('other_language_df.csv', index=False) # CAREFUL, you can overwrite your previous dataset, UPDATE the name when used


# Small Case Text Preprocessing

In [11]:
import numpy as np

df = pd.read_csv('english_df.csv', encoding='utf-8')

# Drop rows with missing values in the 'poem' column
df = df.dropna(subset=['poem'])

In [12]:
print(df.shape)


(44869, 7)


In [13]:
# Pre-compile the patterns for efficiency
url_pattern = re.compile(r'https://\S+|www\.\S+')

chars_to_replace_by_apostrophe = re.compile(r'ÃƒÂ¢Ã¢â€šÂ¬Ã¢â€žÂ¢|Ã¢â‚¬â„¢')
chars_to_replace_by_a = re.compile(r'&aacute')

unwanted_chars_pattern = re.compile(r'Ã¢â‚¬Å|Ã¢â‚¬â„¢|Ã¢â‚¬â€|â‚¬Âº|ÃƒÂ¢Ã¢â€šÂ¬Ã|Ã|¬|¢|š|€||âº|â|Âº|Ã¢|Ãƒâ€|¬ÂºÃƒâ€¦Ã¢â|¬|Å|¾|Âº|Ã¢|Âº|Ã¢â‚¬ÂºÃƒâ€')

def wrangle_text(text):

    # Decode HTML entities
    text = html.unescape(text)

    # Remove HTML tags
    text = BeautifulSoup(text, "lxml").text
    
    # Replace by apostrophe
    text = chars_to_replace_by_apostrophe.sub('´', text)

    # Replace by a
    text = chars_to_replace_by_a.sub('á', text)

    # Fix specific character issues in one go
    
    text = unwanted_chars_pattern.sub(' ', text)

    # Expand contractions once, after all substitutions
    text = contractions.fix(text)
    
    return text

In [15]:
poem = df['poem'].iloc[84] # This is the 84th poem
wrangled_poem = wrangle_text(poem)
print(colored('\n============= First poem in corpus =============', 'blue'))
print(poem)
print(colored('\n============= After wrangling result =============', 'green'))
print(wrangled_poem)


============= First poem in corpus =============
I want to believe IÃ¢â‚¬â„¢m on my last life. What is nirvana if not a kind of death? In a past life, a stranger asked the Buddha for his children. The Buddha offered him his eyes. Then, pulling the children, screaming, from the shivering rice barrels, he gave them away. Did the children ever forgive him? Did they have to because he was the Buddha? I stay awake, listening to my brotherÃ¢â‚¬â„¢s breath as he sleeps in my corridor. A 6 am flight, a half-hug in the parking lot. I want one more life. This is the problem with reincarnation: you donÃ¢â‚¬â„¢t know if what youÃ¢â‚¬â„¢ve lost is lost forever. Unless, I guess, youÃ¢â‚¬â„¢re the Buddha. His children escaped, by the way. Hid from their possessor in the cane grass, the silk reeds, and wove their way back home. I think they hugged their mother. I think the Buddha demanded understanding. I had to, he likely said. I wonder who they became in their next life. I wonder if the siblings st

In [16]:
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import sent_tokenize


print(colored('\n============= First poem in data =============', 'blue'))
print(wrangled_poem)

# Tokenize the review
poem_tokens = wordpunct_tokenize(wrangled_poem)
# Tokenize the review at the sentence level
poem_tokens_sent = sent_tokenize(wrangled_poem)

print(colored('\n============= First poem (tokens) =============', 'green'))
print(poem_tokens)

print(colored('\n============= First poem (tokens sent level) =============', 'red'))
print(poem_tokens_sent)


============= First poem in data =============
I want to believe I´m on my last life. What is nirvana if not a kind of death? In a past life, a stranger asked the Buddha for his children. The Buddha offered him his eyes. Then, pulling the children, screaming, from the shivering rice barrels, he gave them away. Did the children ever forgive him? Did they have to because he was the Buddha? I stay awake, listening to my brother´s breath as he sleeps in my corridor. A 6 am flight, a half-hug in the parking lot. I want one more life. This is the problem with reincarnation: you don´t know if what you´ve lost is lost forever. Unless, I guess, you´re the Buddha. His children escaped, by the way. Hid from their possessor in the cane grass, the silk reeds, and wove their way back home. I think they hugged their mother. I think the Buddha demanded understanding. I had to, he likely said. I wonder who they became in their next life. I wonder if the siblings stayed together, across this ever-short

### Converting into lower case & removing alphanumeric: 

In [17]:
poem_tokens_filtered = [token.lower() for token in poem_tokens if token.isalnum()]

# The isalnum() method returns True if all characters in the string are 
# alphanumeric (either alphabets or numbers). If not, it returns False.

print(colored('\n============= First poem (tokens) =============', 'blue'))
print(poem_tokens)

print(colored('\n============= First poem (tokens filtered) =============', 'green'))
print(poem_tokens_filtered)


============= First poem (tokens) =============
['I', 'want', 'to', 'believe', 'I', '´', 'm', 'on', 'my', 'last', 'life', '.', 'What', 'is', 'nirvana', 'if', 'not', 'a', 'kind', 'of', 'death', '?', 'In', 'a', 'past', 'life', ',', 'a', 'stranger', 'asked', 'the', 'Buddha', 'for', 'his', 'children', '.', 'The', 'Buddha', 'offered', 'him', 'his', 'eyes', '.', 'Then', ',', 'pulling', 'the', 'children', ',', 'screaming', ',', 'from', 'the', 'shivering', 'rice', 'barrels', ',', 'he', 'gave', 'them', 'away', '.', 'Did', 'the', 'children', 'ever', 'forgive', 'him', '?', 'Did', 'they', 'have', 'to', 'because', 'he', 'was', 'the', 'Buddha', '?', 'I', 'stay', 'awake', ',', 'listening', 'to', 'my', 'brother', '´', 's', 'breath', 'as', 'he', 'sleeps', 'in', 'my', 'corridor', '.', 'A', '6', 'am', 'flight', ',', 'a', 'half', '-', 'hug', 'in', 'the', 'parking', 'lot', '.', 'I', 'want', 'one', 'more', 'life', '.', 'This', 'is', 'the', 'problem', 'with', 'reincarnation', ':', 'you', 'don', '´', 't', 'k

In [18]:
stemmer = SnowballStemmer('english')
wnl = WordNetLemmatizer()

stemmed_poem = [stemmer.stem(el) for el in poem_tokens_filtered]
print(colored('\n============= Stemmed review  =============', 'blue'))
print(stemmed_poem)

lemmatized_poem = [wnl.lemmatize(el) for el in poem_tokens_filtered]
print(colored('\n============= Lemmatized review  =============', 'blue'))
print(lemmatized_poem)

# The difference between both is that the lemmatization process is more complex
# and it involves resolving words to their base or root form.

# Lemmatization: Lemmatization is the process of converting a word to its base form.
# Stemming: Stemming is the process of reducing a word to its root form.

# For example: 
# Lemmatization: The word "better" is converted to "good".
# Stemming: The word "better" is converted to "bet".


============= Stemmed review  =============
['i', 'want', 'to', 'believ', 'i', 'm', 'on', 'my', 'last', 'life', 'what', 'is', 'nirvana', 'if', 'not', 'a', 'kind', 'of', 'death', 'in', 'a', 'past', 'life', 'a', 'stranger', 'ask', 'the', 'buddha', 'for', 'his', 'children', 'the', 'buddha', 'offer', 'him', 'his', 'eye', 'then', 'pull', 'the', 'children', 'scream', 'from', 'the', 'shiver', 'rice', 'barrel', 'he', 'gave', 'them', 'away', 'did', 'the', 'children', 'ever', 'forgiv', 'him', 'did', 'they', 'have', 'to', 'becaus', 'he', 'was', 'the', 'buddha', 'i', 'stay', 'awak', 'listen', 'to', 'my', 'brother', 's', 'breath', 'as', 'he', 'sleep', 'in', 'my', 'corridor', 'a', '6', 'am', 'flight', 'a', 'half', 'hug', 'in', 'the', 'park', 'lot', 'i', 'want', 'one', 'more', 'life', 'this', 'is', 'the', 'problem', 'with', 'reincarn', 'you', 'don', 't', 'know', 'if', 'what', 'you', 've', 'lost', 'is', 'lost', 'forev', 'unless', 'i', 'guess', 'you', 're', 'the', 'buddha', 'his', 'children', 'escap',

# Preprocessing of the whole dataset

In [19]:
import numpy as np

df = pd.read_csv('english_df.csv', encoding='utf-8')

# Drop rows with missing values in the 'poem' column
df = df.dropna(subset=['poem'])

In [20]:
print(df.shape)

(44869, 7)


In [21]:
df2 = df.copy()
df2['poem'] = df['poem'].apply(lambda x: wrangle_text(x))

/tmp/ipykernel_1276/2818655410.py:15: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "lxml").text


In [22]:
df2.to_csv('data_wrangled.csv', index=False) # CAREFUL, you can overwrite your previous dataset, UPDATE the name when used

In [23]:
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

In [24]:
# Initialize NLTK resources
stopwords_en = set(stopwords.words('english'))
wnl = WordNetLemmatizer()


def prepare_data(text, tool='lemmatize'):

    # Decode HTML entities
    text = html.unescape(text) 
    
    # Wrangle text
    text = wrangle_text(text)

    # Tokenize text
    tokens = wordpunct_tokenize(text)

    # Filter tokens
    tokens_filtered = [token.lower() for token in tokens if token.isalnum()]

    if tool == 'stem':
        # Stem tokens
        tokens_filtered = [stemmer.stem(el) for el in tokens_filtered]
    elif tool == 'lemmatize':
        # Lemmatize tokens
        tokens_filtered = [wnl.lemmatize(el) for el in tokens_filtered]
    else:
        raise ValueError('Invalid tool. Please select either "stem" or "lemmatize".')

    # Remove stopwords
    clean_tokens = [el for el in tokens_filtered if el not in stopwords_en]

    return clean_tokens

### Processing of the poems column

In [25]:
# Now lets use the original text and check it: 
print(colored('\n============= First review in corpus =============', 'blue'))
poem = df['poem'].iloc[84]
print(poem)

# Prepare the data
clean_tokens = prepare_data(poem, tool = "lemmatize")
print(colored('\n============= Clean tokens =============', 'green'))
print(clean_tokens)



============= First review in corpus =============
I want to believe IÃ¢â‚¬â„¢m on my last life. What is nirvana if not a kind of death? In a past life, a stranger asked the Buddha for his children. The Buddha offered him his eyes. Then, pulling the children, screaming, from the shivering rice barrels, he gave them away. Did the children ever forgive him? Did they have to because he was the Buddha? I stay awake, listening to my brotherÃ¢â‚¬â„¢s breath as he sleeps in my corridor. A 6 am flight, a half-hug in the parking lot. I want one more life. This is the problem with reincarnation: you donÃ¢â‚¬â„¢t know if what youÃ¢â‚¬â„¢ve lost is lost forever. Unless, I guess, youÃ¢â‚¬â„¢re the Buddha. His children escaped, by the way. Hid from their possessor in the cane grass, the silk reeds, and wove their way back home. I think they hugged their mother. I think the Buddha demanded understanding. I had to, he likely said. I wonder who they became in their next life. I wonder if the siblings 

In [26]:
# Apply the prepare_data function to clean the poems
df['clean_tokens'] = df['poem'].apply(lambda x: prepare_data(x, tool='lemmatize'))

/tmp/ipykernel_1276/2818655410.py:15: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "lxml").text


In [27]:
# remove the column 'poem'
df = df.drop(columns=['poem'])

In [29]:
print(df.shape)

(44869, 7)


In [30]:
df.to_csv('data_poems_tokenized.csv', index=False) # CAREFUL, you can overwrite your previous dataset, UPDATE the name when used

In [33]:
df.head(10)

,id,title,author,snippet,link,categories,clean_tokens
0,162275,The 80&rsquo;s Miracle Diet,By Melvin Dixon,Yours free without the asking,https://www.poetryfoundation.org/poetrymagazin...,"['Living', 'Health & Illness']","[free, without, asking, quick, delivery, via, ..."
1,162250,All Saints&rsquo;,By Corey Van Landingham,CaravaggioÃ¢â‚¬â„¢s face in the sunken pumpkin.,https://www.poetryfoundation.org/poetrymagazin...,[],"[caravaggio, face, sunken, pumpkin, bulb, rott..."
2,162279,And These Are Just a Few ...,By Melvin Dixon,This poem is for the epidemic dead and the liv...,https://www.poetryfoundation.org/poetrymagazin...,"['Living', 'Health & Illness', 'Social Comment...","[poem, epidemic, dead, living, remember, neigh..."
3,162248,ASMR,By Corey Van Landingham,Why not climb up the mountain,https://www.poetryfoundation.org/poetrymagazin...,[],"[climb, mountain, delight, world, thin, meridi..."
4,162249,ASMR,By Corey Van Landingham,HelloÃ¢â‚¬â€Tonight,https://www.poetryfoundation.org/poetrymagazin...,[],"[hello, tonight, trace, static, bough, tempora..."
5,162276,Autumn Leaving,By Melvin Dixon,"Vermont leaves under glass,",https://www.poetryfoundation.org/poetrymagazin...,[],"[didier, vermont, leaf, glass, hand, working, ..."
6,162292,Babble,By Nica Giromini,Say under land,https://www.poetryfoundation.org/poetrymagazin...,[],"[say, land, remembered, muddy, rush, slows, ne..."
7,162295,Chiaroscuro Spring time,By Cyrus Cassells,That was the season I couldn´„¢t think or writ...,https://www.poetryfoundation.org/poetrymagazin...,"['Arts & Sciences', 'Painting & Sculpture']","[galleria, nazionale, arte, antica, rome, wa, ..."
8,162291,Currency,By Nica Giromini,As in a net. Headfirst,https://www.poetryfoundation.org/poetrymagazin...,[],"[net, headfirst, prairie, spring, seeing, noth..."
9,162267,Everyday around the world a woman is pulled in...,By Krista Franklin,Everyday some brown,https://www.poetryfoundation.org/poetrymagazin...,"['Types/Modes', 'Ekphrasis']","[kara, walker, blue, â, 2020, everyday, brown,..."
